# 8. 딥러닝

# 8.1 더 깊게

## 8.1.1 더 깊은 신경망으로

- 그림8.1 손글씨 숫자를 인식하는 심층 CNN  
<img src='images/fig 8-1.png' width=700>  

- 3*3의 작은 필터를 사용한 합성곱 계층
- 층이 깊어질수록 채널 수가 늘어남
- 활성화 함수는 ReLU
- 완전연결 계층 뒤에 드롭아웃 계층 사용
- Adam을 사용해 최적화
- 가중치 초깃값은 He의 초깃값

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient
from dataset.mnist import load_mnist
from common.trainer import Trainer
import pickle


class DeepConvNet:
    """정확도 99% 이상의 고정밀 합성곱 신경망

    네트워크 구성은 아래와 같음
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 가중치 초기화===========
        # 각 층의 뉴런 하나당 앞 층의 몇 개 뉴런과 연결되는가（TODO: 자동 계산되게 바꿀 것）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        weight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = weight_init_scales[idx] * np.random.randn(conv_param['filter_num'], 
                                                        pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = weight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = weight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 계층 생성===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

In [ ]:
from dataset.mnist import load_mnist
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                 epochs=20, mini_batch_size=100,
                 optimizer='Adam', optimizer_param={'lr':0.001},
                 evaluate_sample_num_per_epoch=1000)
trainer.train()

In [6]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()
# 학습에 긴 시간이 걸리므로 미리 학습한 결과를 pkl로 배포
network.load_params(r'.\deeplearning_from_scratch-master\ch8.딥러닝\deep_convnet_params.pkl')

print(network.accuracy(x_test, t_test))  # 0.9935

0.9935


- 그림8.2 인식하지 못한 이미지들 : 각 사진의 왼쪽위는 정답 레이블, 오른쪽 아래는 이 신경망의 추론 결과  
<img src='images/fig 8-2.png' width=700>  

## 8.1.2 정확도를 더 높이려면

- NOTE : MNIST는 상대적으로 단순하기 때문에 층을 깊게 하지 않고도 좋은 결과가 나오지만
- 대규모 일반 사물 인식에 대해서는 층을 깊게 하면 얻는 정확도가 크다.

- 데이터 확장data augmentation : 훈련 이미지를 알고리즘을 이용해 인위적으로 늘린다.
- 이미지 회전, 이동, crop, flip, 확대, 축소, 밝기 등을 변경한다.

- Data augmentation
    - 회전, 이동, crop, flip 등을 통해서 데이터의 양을 늘리는 기법을 의미한다.

- 그림8.4 데이터 확장의 예  
<img src='images/fig 8-4.png' width=700>  

## 8.1.3 깊게 하는 이유

- 학습의 효율성이 좋아진다.
    - 예를 들어 5*5 receptive field를 분석하기 위해서는 한층을 사용할 경우에는 25개의 파라미터가 필요하다. 하지만 두 층을 사용하면 18개의 파라미터만 학습시키면 된다.

- 그림8.5 5X5  합성곱 연산의 예  
<img src='images/fig 8-5.png' width=700>  

- 그림8.6 3X3  합성곱 연산의 예  
<img src='images/fig 8-6.png' width=700>  

- 계층적으로 분석할 수 있다.
    - 예를들어 이미지를 보고, 클래스를 예측하는 분류문제에서 한층으로 한다고 하면 이미지에서 바로 어떤 클래스인지 맞춰야한다. 하지만 여러 층으로 나누게 되면 첫 번째 층에는 엣지나 blob 같은 간단한 특징을 뽑고, 층이 깊어질수록 조금 더 복잡한 특징을 추출하는 등 계층적으로 분석을 할 수 있다.

# 8.2 딥러닝의 초기 역사

- 딥러닝이 주목받은 계기는 이미지 인식 대회 ILSVRC의 2012년 대회이다.
- AlexNet이 기존 대비 압도적 성적(오류율 25.8%->16.4%)으로 우승한 이후 딥러닝이 주류가 됐다.

## 8.2.1 이미지넷

- 이미지넷은 100만장이 넘는 이미지를 담고 있는 데이터셋이다. 이 데이터를 이용한 시험 중 하나가 분류대회이다. 분류대회에서는 1000개의 클래스를 제대로 분류하는지를 겨룬다.

- 그림8.7 대규모 데이터 ImagNet의 데이터들  
<img src='images/fig 8-7.png' width=1000>  

- 그림8.8 ILSVRC 최우수 팀의 성적 추이  
<img src='images/fig 8-8.png' width=1000>  

## 8.2.2 VGG

- 합성곱 계층과 풀링 계층으로 구성되는 기본적인 CNN. 16층(19층) 신경망을 사용한다.
- 2014년 대회 2위에 올랐으며 구성이 간단해 응용하기 좋아 많이 사용된다.
- 16층으로 층 수를 이전보다 늘렸고, 3*3의 작은 필터를 사용한 합성곱 계층을 연속으로 거친다.

- 그림8.9 VGG  
<img src='images/fig 8-9.png' width=1000>  

## 8.2.3 GoogLeNet

- 깊이 뿐 아니라 폭의 너비도 깊다는 것이 특징(인셉션 구조) 크기가 다른 여러 필터와 풀링을
필터로 결합한다.

- 그림8.10 GoogLeNet  
<img src='images/fig 8-10.png' width=1000>  

- GoogLeNet은 가로 방향에 폭이 있고, 이를 인셉션 구조라한다. 인셉션 구조는 크기가 다른 필터를 여러 개 적용하여 그 결과를 결합한다.

- 그림8.11 GoogLeNet의 인셉션 구조  
<img src='images/fig 8-11.png' width=1000>  

## 8.2.4 ResNet

- Residual Netwrk 마이크로소프트가 개발한 네트워크
층이 지나치게 깊으면 학습이 잘 되지 않고 성능이 떨어지는 문제를 해결하기 위해 VGG 신경망 기반에
스킵 연결을 도입했다. 
- 입력 데이터를 일부 합성곱 계층을 건너뛰어 출력에 바로 더한다.
역전파 때 신호 감쇠를 막아준다.

- 스킵연결은 입력데이터를 합성곱 계층을 건너뛰어 출력에 바로 더하는 구조이다.

- 그림8.12 ResNet의 구성요소 : weight layer는 합성곱 계층을 말한다.  
<img src='images/fig 8-12.png' width=700>  

- 전이 학습transfer learning 
    - 학습된 가중치를 다른 신경망에 복사한 다음 재학습을 수행하는 것. 보유한 데이터셋이 적을 때 유용하다.

- 그림8.13 ResNet : 블록이 3X3인 합성곱 계층에 대응, 층을 건너뛰는 스킵 연결이 특징이다.  
<img src='images/fig 8-13.png' width=1000>  

- ILSVRC 대회에서 톱-5 오류율이 3.5%라는 경이적인 결과를 냈음

# 8.3 더 빠르게(딥러닝 고속화)

- GPU를 활용해 대량의 연산을 고속으로 처리할 수 있다. 최근 프레임워크에서는 학습을 복수의 GPU와 여러 기기로 분산 수행한다.

## 8.3.1 풀어야 할 숙제

- AlexNet의 순전파에서 각 층이 소비하는 시간은 GPU의 95%, CPU의 89%가 합성곱 계층이다.
- 결국 단일 곱셈-누산을 어떻게 효율적으로 계산하느냐는 문제가 된다.

- 그림8.14 AlexNet의 forward 처리 시 각 층의 시간 비율  
<img src='images/fig 8-14.png' width=700>  

## 8.3.2 GPU를 활용한 고속화

- 알렉스넷의 학습시간은 CPU에서는 40일이나 걸리지만 GPU로는 6일까지 단축된다. 
- cuDNN이라는 딥러닝에 최적화된 라이브러리를 사용하면 더 빨라진다.

- 그림8.15 AlexNet의 학습시간을 '16코어 제온CPU'와 엔비디아의 '타이탄 GPU'에서 비교한 결과  
<img src='images/fig 8-15.png' width=700>  

## 8.3.3 분산 학습

- 1회 학습에 걸리는 시간을 단축하고 싶다는 요구에서 딥러닝 학습을 수평 확장하자는 아이디어(=분산학습)가 생겨나게 되었다. 구글의 텐서플로와 마이크로소프트의 CNTK(Computational Network Toolkit)는 다수의 GPU와 컴퓨터를 이용한 분산 학습을 지원한 딥러닝 프레임워크들이다. 분산학습에서는 "계산을 어떻게 분산시키느냐"가 어려운 문제이다. 컴퓨터 사이의 통신과 데이터 동기화 등의 문제를 가지고 있기 때문에, 텐서플로와 같은 프레임워크를 사용하는게 좋으며 이런 프레임워크들은 분산학습에 역점을 두고 개발되고 있다.

- 그림8.16 텐서플로의 분산 학습 성능  
<img src='images/fig 8-16.png' width=700>  

## 8.3.4 연산 정밀도와 비트 줄이기

- 계산 능력 외에도 메모리 용량과 버스 대역폭등을 고려하면 네트워크로 주고받는 데이터의 비트수는 최소로 만드는 것이 바람직 할 것이다. 메모리 용량의 경우, 대량의 가중치 매개변수와 중간 데이터를 메모리에 저장해야함을 고려해야한다. 또한, 버스 대역폭의 경우, 버스를 흐르는 데이터가 많아 한계를 넘어서게 된다면 병목이 발생하게 된다. 

- 딥러닝은 높은 수치 정밀도를 요구하진 않는다. 지금까지의 실험으로는 딥러닝은 16비트 반정밀도만 사용해도 학습에 문제가 없다고 알려져 있다.  딥러닝의 비트 수를 줄이는 기술은 특히 딥러닝을 임베디드용으로 이용할 때 중요한 주제이다.

    caluculate accuracy (float64) ...  
    0.9935  
    caluculate accuracy (float16) ...  
    0.9935  

# 8.4 딥러닝의 활용

## 8.4.1 사물 검출

- 이미지 속에 담긴 사물의 위치와 종류(클래스)를 알아냄

- 그림8.17 사물 검출의 예  
<img src='images/fig 8-17.png' width=700>  

- 사물의 위치와 다수의 사물이 존재할 수 있기 때문에 사물 인식보다 어려운 문제.
- R-CNN(Regions with CNN)이 유명하다. 후보 영역 추출 후 각 영역에 CNN을 적용하여 분류한다.
- 후보 영역 추출은 기존 컴퓨터 비전 분야의 기법을 사용할 수 있고 이 부분까지 CNN으로 처리하는
Faster R-CNN 기법도 있다.

- 그림8.18 R-CNN의 처리 흐름  
<img src='images/fig 8-18.png' width=700>  

## 8.4.2 분할

- 이미지를 픽셀 수준에서 분류하는 문제
- 픽셀 단위로 객체마다 채색된 지도 데이터를 사용해 학습함, 추론할 때 입력 이미지의 모든 픽셀을 분류
- 픽셀 수만큼 forward 처리를 해야 하여 긴 시간이 걸리게 됨

- 그림8.19 분할의 예 : 왼쪽이 입력 이미지, 오른쪽이 지도용 이미지  
<img src='images/fig 8-19.png' width=700>  

- FCN - 단 한 번의 forward 처리로 모든 픽셀의 클래스를 분류해줌, 합성곱 계층만으로 구성된 네트워크
- 공간 볼륨을 유지한 채 마지막 출력까지 처리 가능, 이중 선형 보간에 의한 선형 확대
- 일반적인 CNN이 완전연결 계층을 이용하는 반면 FCN에서는 이를 '같은 기능을 하는 합성곱 계층'으로 바꾼다. 마지막 출력까지 공간 볼륨을 유지한 채 처리할 수 있다.

- 입력 크기가 32X10X10인 데이터의 완전 연결 계층은 필터 크기가 32X10X10인
합성곱 계층으로 대체할 수 있다.
- 완전연결 계층의 출력 노드가 100개라면 이러한 필터 100개를 사용하면 된다.

- 그림8.20 FCN의 전체 그림  
<img src='images/fig 8-20.png' width=700>  

## 8.4.3 사진 캡션 생성

- NIC(Neural Image Caption) 모델이 대표적. 심층 CNN과 자연어를 다루는 순환 신경망(RNN)으로 구성된다.
- 멀티모달 처리 : 사진이나 자연어 같은 여러 종류의 정보를 조합하고 처리하는 것.
- NOTE : RNN은 신경망이 순환적 네트워크 구조를 가진다. 이로 인해 이전에 생성한 정보에 영향을
받는 점이 RNN의 특징이다. 자연어와 시계열 데이터 등 연속성 있는 데이터를 다룰 때 사용한다.

- 그림8.21 딥러닝으로 사진 캡션을 생성하는 예  
<img src='images/fig 8-21.png' width=700>  

- 그림8.22 NIC의 전체 구성  
<img src='images/fig 8-22.png' width=700>  

## 8.5 딥러닝의 미래

## 8.5.1 이미지 스타일 변환

- 그림을 그리는 연구
- 두 이미지를 입력해서 새로운 그림을 생성하는 연구
- 네트워크의 중간 데이터가 콘텐츠 이미지의 중간 데이터와 비슷해지도록 학습. 콘텐츠 이미지의 형태를 흉내냄.
- 스타일 행렬- 스타일 이미지의 화풍을 흡수하기 위해 사용

- 그림8.23 A Neural Algorithm of Artistic Style 
<img src='images/fig 8-23.png' width=700>  

## 8.5.2 이미지 생성

- 이미지 입력 없이 새로운 이미지를 그려내는 연구.
- DCGAN(Deep Convolutional Generative Adversarial Network) 기법을 통해 학습 데이터에는
존재하지 않는 새로운 침실 이미지들을 생성했다.

- DCGAN은 생성자Generator와 식별자Discriminator라 불리는 2개의 신경망을 이용한다.
- 생성자가 이미지를 생성하고 식별자는 진짜인지를 판별한다. 
- 둘을 겨루도록 학습시켜 생성자는
- 더 정교한 이미지 생성을, 식별자는 더 정확한 판별을 하게 된다.
- 이런 개념을 GAN(Generative Adversarial Network)이라 한다.

- 그림8.24 DC GAN으로 새롭게 생성한 침실 이미지들 
<img src='images/fig 8-24.png' width=1000>  

- NOTE : 이전까지의 기계학습은 지도 학습supervised learning이라는 유형의 문제였다.
하지만 이번 절의 문제는 지도용 데이터는 주어지지 않고 대량의 이미지만 주어진다.
즉, 지도 없이 스스로 학습하는 자율 학습 unsupervised learning 문제이다.

## 8.5.3 자율 주행

- 경로 계획과 센서 기술 등이 있지만 가장 중요한 기술은 주위 환경을 올바르게 인식하는 것.
- SegNet이라는 CNN기반 신경망은 입력 이미지를 픽셀 수준에서 인식할 수 있다.

- 그림8.25 딥러닝을 활용한 이미지 분할의 예: 도로, 차, 건물, 인도 등을 정확하게 인식한다. 
<img src='images/fig 8-25.png' width=1000>  

## 8.5.4 Deep Q-Network(강화학습)

- 에이전트가 환경에 맞게 행동을 선택하고, 그 행동에 의해서 환경이 변함
- 환경이 변화하면 에이전트를 보상을 얻음. 더 나은 보상을 받는 쪽으로 에이전트의 행동 지침을 바로잡음

- 그림8.26 강화학습의 기본 틀 : 에이전트는 더 좋은 보상을 받기 위해 스스로 학습한다. 
<img src='images/fig 8-26.png' width=500>   

- Deep Q-Network - Q 학습이라는 강화학습 알고리즘을 기초로 함
- Q 학습 - 최적 행동 가치 함수로 최적인 행동을 정함. 
- DQN - Q 학습을 딥러닝을 비슷하게 흉내내어 사용함

- 그림8.27 Deep Q-Network로 비디오 게임 조작을 학습한다.   
<img src='images/fig 8-27.png' width=1000> 

# 8.6 정리

- 수많은 문제에서 신경망을 더 깊게 하여 성능을 개선할 수 있다.
- 이미지 인식 기술 대회인 ILSVRC에서는 최근 딥러닝 기반 기법이 상위권을 독점하고 있으며, 그 깊이도
더 깊어지는 추세다.
- 유명한 신경망으로는 VGG, GoogLeNet, ResNet이 있다.
- 딥러닝(신경망)은 사물 인식 뿐만 아니라 사물 검출과 분할에도 이용할 수 있다.
- 딥러닝의 응용 분야로는 사진의 캡션 생성, 이미지 생성, 강화학습등이 있다. 최근에는 자율 주행에도 딥러닝을 접목하고 있어 기대된다.